In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"
load_sequence_aggregation_path = analysis_path + "load_sequence_agg/"
results_aggregation_path = analysis_path + "results_to_aggregate/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)

Using Theano backend.


after shuffling: [('sequence_2b_445_6_fv1c.npy', 'sequence_2b_445_6_label_fv1c.npy'), ('sequence_1b_12_3_fv1c.npy', 'sequence_1b_12_3_label_fv1c.npy'), ('sequence_1b_55_6_fv1c.npy', 'sequence_1b_55_6_label_fv1c.npy'), ('sequence_3a_689_11_fv1c.npy', 'sequence_3a_689_11_label_fv1c.npy'), ('sequence_1c_101_2_fv1c.npy', 'sequence_1c_101_2_label_fv1c.npy'), ('sequence_1k_297_8_fv1c.npy', 'sequence_1k_297_8_label_fv1c.npy'), ('sequence_1k_281_2_fv1c.npy', 'sequence_1k_281_2_label_fv1c.npy'), ('sequence_1l_363_4_fv1c.npy', 'sequence_1l_363_4_label_fv1c.npy'), ('sequence_1l_375_6_fv1c.npy', 'sequence_1l_375_6_label_fv1c.npy'), ('sequence_1d_218_29_fv1c.npy', 'sequence_1d_218_29_label_fv1c.npy'), ('sequence_1d_234_15_fv1c.npy', 'sequence_1d_234_15_label_fv1c.npy'), ('sequence_1d_162_3_fv1c.npy', 'sequence_1d_162_3_label_fv1c.npy'), ('sequence_1d_120_1_fv1c.npy', 'sequence_1d_120_1_label_fv1c.npy'), ('sequence_1d_194_5_fv1c.npy', 'sequence_1d_194_5_label_fv1c.npy'), ('sequence_1l_392_3_fv1c.npy

In [2]:
results_files_list = list(os.listdir(results_aggregation_path))
results_files_list.sort()
metrics_list = ['mae','mape','mse']
i=0
for files in test_set_filenames:
    #print("filename", files)
    i += 1
    temp_results_dict={}
    temp_results_df = pd.DataFrame(columns=metrics_list)
    data_load_path = test_path + '/data/' + files[0]
    label_load_path = test_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
#     test_array = np.load(data_load_path)
#     test_label_array = np.load(label_load_path)[:, 1:]
    
    #nested for but whatever
    for results_file in results_files_list:
        print("analyzing ", str(files[0][:-4]), "with model ", str(results_file))
        complete_filename = results_aggregation_path + results_file
        interm_results_df = pd.read_csv(complete_filename,index_col='seq_name')
        #print(interm_results_df.index)
        print("does it have .npy in the results?", interm_results_df.index.contains("sequence_1c_89_10_fv1c.npy"))
        
        suffix_check = interm_results_df.index.contains("sequence_1c_89_10_fv1c.npy")
        if suffix_check == True:
            seqname = str(files[0])
        if suffix_check == False:
            seqname = str(files[0])[:-4]
            
        temp_results_dict[str(results_file)]=interm_results_df.loc[seqname,:]
            
        for metric in list(interm_results_df.columns): #.remove('filename'): #make it adaptive to whatever results 
            if metric in temp_results_df.columns:
                temp_results_df.loc[:,[metric]] = interm_results_df.loc[seqname,[metric]]
        
    temp_results_df = pd.DataFrame.from_dict(temp_results_dict,orient='index')
    temp_results_df.sort_values(by=['mae'],inplace=True)
    temp_results_df.to_csv(load_sequence_aggregation_path + str(files[0][:-4]) + ".csv")

('analyzing ', 'sequence_3a_608_10_fv1c', 'with model ', 'combi_scores_elasticnet__bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_608_10_fv1c', 'with model ', 'combi_scores_elasticnet_model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_608_10_fv1c', 'with model ', 'combi_scores_et30_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_608_10_fv1c', 'with model ', 'combi_scores_et5_ws__model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_608_10_fv1c', 'with model ', 'combi_scores_fv1c_elasticnet_50sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_608_10_fv1c', 'with model ', 'combi_scores_f

('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_fv1clinreg_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_fv1crf60_ws__.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_fv1cridgecholesky_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_fv1cridgesaga_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_linreg_model__tiny_bidir_1sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_rf30_ws__model__tiny_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_687_9_fv1c', 'with model ', 'combi_scores_rf5_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_s

('analyzing ', 'sequence_1k_348_9_fv1c', 'with model ', 'scores_conv__tree_tinybidir_nodense_fv1c_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_348_9_fv1c', 'with model ', 'scores_conv_lstm_bagged__bag_conv_lstm_dense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_348_9_fv1c', 'with model ', 'scores_lstm_rerun__bidir100ea_fv1c_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_348_9_fv1c', 'with model ', 'scores_lstm_rerun__big_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_348_9_fv1c', 'with model ', 'scores_lstm_rerun__tiny_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_348_9_fv1c', 'with model ', 'scores_sk_conv_lstm_bagged__bag_conv_lstm_dense_tiny_shufstart_relu_ca_tanh_da_

('analyzing ', 'sequence_3a_667_9_fv1c', 'with model ', 'scores_sk_conv_lstm_bagged__bag_conv_lstm_dense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_667_9_fv1c', 'with model ', 'scores_sk_conv_lstm_bagged__bag_conv_lstm_nodense_micro_128d_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1d_139_10_fv1c', 'with model ', 'combi_scores_elasticnet__bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1d_139_10_fv1c', 'with model ', 'combi_scores_elasticnet_model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1d_139_10_fv1c', 'with model ', 'combi_scores_et30_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tan

('analyzing ', 'sequence_2c_539_10_fv1c', 'with model ', 'combi_scores_rf30_ws__model__tiny_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_2c_539_10_fv1c', 'with model ', 'combi_scores_rf5_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_2c_539_10_fv1c', 'with model ', 'combi_scores_ridgesaga_model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_2c_539_10_fv1c', 'with model ', 'scores_conv__tree_medbidir_nodense_fv1c_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_2c_539_10_fv1c', 'with model ', 'scores_conv__tree_tinybidir_nodense_fv1c_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence

...

('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_329_10_fv1c', 'with model ', 'scores_lstm_rerun__big_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_329_10_fv1c', 'with model ', 'scores_lstm_rerun__tiny_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_329_10_fv1c', 'with model ', 'scores_sk_conv_lstm_bagged__bag_conv_lstm_dense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_329_10_fv1c', 'with model ', 'scores_sk_conv_lstm_bagged__bag_conv_lstm_nodense_micro_128d_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_2b_489_10_fv1c', 'with model ', 'combi_scores_elasticnet__bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have

('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_fv1clinreg_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_fv1crf60_ws__.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_fv1cridgecholesky_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_fv1cridgesaga_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_linreg_model__tiny_bidir_1sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_rf30_ws__model__tiny_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_3a_668_10_fv1c', 'with model ', 'combi_scores_rf5_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_

('does it have .npy in the results?', False)
('analyzing ', 'sequence_1c_89_10_fv1c', 'with model ', 'combi_scores_elasticnet_model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1c_89_10_fv1c', 'with model ', 'combi_scores_et30_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1c_89_10_fv1c', 'with model ', 'combi_scores_et5_ws__model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1c_89_10_fv1c', 'with model ', 'combi_scores_fv1c_elasticnet_50sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1c_89_10_fv1c', 'with model ', 'combi_scores_fv1clinreg_50sd.csv')
('does it have .npy in the results?', True)
('analyzing ', 'sequence_1c_89_10_fv1c', 'with model ', 'combi_scores_fv1crf60_ws__.csv')
('does it have .npy in the results?', True)


('analyzing ', 'sequence_1l_438_9_fv1c', 'with model ', 'combi_scores_rf30_ws__model__tiny_bidir_.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1l_438_9_fv1c', 'with model ', 'combi_scores_rf5_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1l_438_9_fv1c', 'with model ', 'combi_scores_ridgesaga_model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1l_438_9_fv1c', 'with model ', 'scores_conv__tree_medbidir_nodense_fv1c_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1l_438_9_fv1c', 'with model ', 'scores_conv__tree_tinybidir_nodense_fv1c_slowlr_relu_ca_sigmoid_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1l_4

('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_319_10_fv1c', 'with model ', 'combi_scores_elasticnet__bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_319_10_fv1c', 'with model ', 'combi_scores_elasticnet_model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_319_10_fv1c', 'with model ', 'combi_scores_et30_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_319_10_fv1c', 'with model ', 'combi_scores_et5_ws__model__tiny_bidir_20sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k_319_10_fv1c', 'with model ', 'combi_scores_fv1c_elasticnet_50sd.csv')
('does it have .npy in the results?', False)
('analyzing ', 'sequence_1k

In [3]:
#load the master_results file
master_results_df = pd.read_csv(analysis_path+"master_results_per_model.csv",index_col=0) #this file is probably created manually...
#declare a list of valid models, cutoff criteria mae < 1
model_ok_list = []
#if it meets the threshold, add to the list
for item in master_results_df.index:
    if master_results_df.loc[item,'mae_avg'] <= 0.50:
        model_ok_list.append(str(item))
#add an if statement to the

In [4]:
print("model ok list: ",model_ok_list, "length", len(model_ok_list))

('model ok list: ', ['combi_scores_elasticnet__bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv', 'combi_scores_elasticnet_model__tiny_bidir_20sd.csv', 'combi_scores_rf30_ws__model__tiny_bidir_.csv', 'combi_scores_ridgesaga_model__tiny_bidir_20sd.csv', 'combi_scores_et5_ws__model__tiny_bidir_20sd.csv', 'combi_scores_linreg_model__tiny_bidir_1sd.csv', 'combi_scores_rf5_ws___bag_convdblfilters_lstm_nodense_tiny_shufstart_relu_ca_tanh_da_3_cbd_standard_per_batch_sclr_l1l2_kr_HLR.csv', 'combi_scores_fv1crf60_ws__.csv', 'combi_scores_fv1cridgesaga_50sd.csv', 'scores_lstm_rerun__tiny_bidir_.csv', 'scores_lstm_rerun__big_bidir_.csv', 'scores_lstm_rerun__bidir100ea_fv1c_.csv', 'combi_scores_fv1clinreg_50sd.csv'], 'length', 13)


In [5]:
main_colnames = ['seq_name','mse_avg','mse_med','mse_stdev','mae_avg','mae_med','mae_stdev','mae_normd_dev','mape_avg','mape_med','mape_stdev','msle_avg','msle_med','msle_stdev']
# mse_main_df = pd.DataFrame(columns=mse_main_colnames)
# mse_main_df.set_index(keys=['results_filename'], inplace=True)


sequence_results_per_model_list = list(os.listdir(load_sequence_aggregation_path))
#still need subset per model category... 
master_per_loadseq_df = pd.DataFrame(columns=main_colnames)
master_per_loadseq_df.set_index(keys=['seq_name'], inplace=True)

interm_per_loadseq_result_modeloksubset = pd.DataFrame(index=['model_name'])
#@@@@@@@@@@@@@@@@@@@@@@@@@@@
metrics_to_aggregate = ['mae','mse','mape','msle']
#@@@@@@@@@@@@@@@@@@@@@@@@@@@
for seq_results in sequence_results_per_model_list:
    interm_per_loadseq_result_df = pd.read_csv(load_sequence_aggregation_path + str(seq_results),index_col=0)
    interm_per_loadseq_result_modeloksubset = pd.DataFrame(index=['model_name']) 
    #set the index to be called model_name
    interm_per_loadseq_result_df.index.rename(name = 'model_name',inplace=True)
    
    #print('interm_per_loadseq_result_df columns',interm_per_loadseq_result_df.columns)
    
    #master_per_loadseq_df[str(seq_results)]
    for metric in metrics_to_aggregate:
        key_avg_colname = str(metric) + "_" + "avg"
        key_med_colname = str(metric) + "_" + "med"
        key_std_colname = str(metric) + "_" + "stdev"
        #print(interm_per_loadseq_result_df.index.isin([model_ok_list]))
        #print(interm_per_loadseq_result_df.loc[model_ok_list,'mae'])
        #for model in model_ok_list:
            #rint("model: ",model, "seq_results: ",seq_results) #needs two aggregations.. #subset models,THEN go.
        print("the mean", interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True).values)
        print("the type of mean", type(interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True)))
        print("the shape of mean", (interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True)).shape)
        master_per_loadseq_df.loc[str(seq_results)[:-4],[key_avg_colname]] = interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].mean(skipna=True).values
        master_per_loadseq_df.loc[str(seq_results)[:-4],[key_med_colname]] = interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].median(skipna=True).values
        master_per_loadseq_df.loc[str(seq_results)[:-4],[key_std_colname]] = interm_per_loadseq_result_df.loc[model_ok_list,[str(metric)]].std(skipna=True).values
        
        print("the master df's entry:",master_per_loadseq_df.loc[str(seq_results)[:-4],[key_avg_colname]])
master_per_loadseq_df.sort_values(by=['mae_avg'],inplace=True)        
master_per_loadseq_df.to_csv(analysis_path + 'master_score_per_loadseq.csv')

('the mean', array([ 0.13648651]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.136487
Name: sequence_2b_449_10_fv1c, dtype: object)
('the mean', array([ 0.08633889]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0863389
Name: sequence_2b_449_10_fv1c, dtype: object)
('the mean', array([ 214.88098251]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    214.881
Name: sequence_2b_449_10_fv1c, dtype: object)
('the mean', array([ 0.05058795]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.050588
Name: sequence_2b_449_10_fv1c, dtype: object)
('the mean', array([ 0.1399199]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's en

("the master df's entry:", msle_avg    0.0648045
Name: sequence_1k_338_9_fv1c, dtype: object)
('the mean', array([ 0.26803347]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.268033
Name: sequence_3a_688_10_fv1c, dtype: object)
('the mean', array([ 0.32492579]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.324926
Name: sequence_3a_688_10_fv1c, dtype: object)
('the mean', array([ 864.56358202]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    864.564
Name: sequence_3a_688_10_fv1c, dtype: object)
('the mean', array([ 0.11426505]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.114265
Name: sequence_3a_688_10_fv1c, dtype: object)
('the mean', array([ 0.27793131]))
('the type o

("the master df's entry:", msle_avg    0.0905505
Name: sequence_2c_549_10_fv1c, dtype: object)
('the mean', array([ 0.30412616]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.304126
Name: sequence_1c_119_10_fv1c, dtype: object)
('the mean', array([ 0.3143671]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.314367
Name: sequence_1c_119_10_fv1c, dtype: object)
('the mean', array([ 512.23705944]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    512.237
Name: sequence_1c_119_10_fv1c, dtype: object)
('the mean', array([ 0.11121838]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.111218
Name: sequence_1c_119_10_fv1c, dtype: object)
('the mean', array([ 0.23520058]))
('the type o

('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0875269
Name: sequence_1k_308_9_fv1c, dtype: object)
('the mean', array([ 231.11410899]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    231.114
Name: sequence_1k_308_9_fv1c, dtype: object)
('the mean', array([ 0.05183038]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.0518304
Name: sequence_1k_308_9_fv1c, dtype: object)
('the mean', array([ 0.18984284]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.189843
Name: sequence_1d_228_9_fv1c, dtype: object)
('the mean', array([ 0.14484659]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.144847
Name: sequence_1d_228_9_fv1c, dtype: object)
('the mean', array([ 45

("the master df's entry:", mae_avg    0.277852
Name: sequence_2b_498_9_fv1c, dtype: object)
('the mean', array([ 0.267361]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.267361
Name: sequence_2b_498_9_fv1c, dtype: object)
('the mean', array([ 476.4482272]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    476.448
Name: sequence_2b_498_9_fv1c, dtype: object)
('the mean', array([ 0.10431691]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.104317
Name: sequence_2b_498_9_fv1c, dtype: object)
('the mean', array([ 0.23453213]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.234532
Name: sequence_3a_568_9_fv1c, dtype: object)
('the mean', array([ 0.27209223]))
('the type of mean', 

...

("the master df's entry:", mse_avg    0.085796
Name: sequence_2b_469_10_fv1c, dtype: object)
('the mean', array([ 202.92980496]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    202.93
Name: sequence_2b_469_10_fv1c, dtype: object)
('the mean', array([ 0.05135564]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.0513556
Name: sequence_2b_469_10_fv1c, dtype: object)
('the mean', array([ 0.18612566]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.186126
Name: sequence_1l_429_10_fv1c, dtype: object)
('the mean', array([ 0.14048983]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.14049
Name: sequence_1l_429_10_fv1c, dtype: object)
('the mean', array([ 565.14741785]))
('the type o

("the master df's entry:", mae_avg    0.136817
Name: sequence_1b_29_10_fv1c, dtype: object)
('the mean', array([ 0.08658526]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0865853
Name: sequence_1b_29_10_fv1c, dtype: object)
('the mean', array([ 165.1012945]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    165.101
Name: sequence_1b_29_10_fv1c, dtype: object)
('the mean', array([ 0.05122911]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.0512291
Name: sequence_1b_29_10_fv1c, dtype: object)
('the mean', array([ 0.20804256]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.208043
Name: sequence_1k_328_9_fv1c, dtype: object)
('the mean', array([ 0.16075598]))
('the type of mea

('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.186977
Name: sequence_1l_438_9_fv1c, dtype: object)
('the mean', array([ 0.14274406]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.142744
Name: sequence_1l_438_9_fv1c, dtype: object)
('the mean', array([ 387.33662915]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    387.337
Name: sequence_1l_438_9_fv1c, dtype: object)
('the mean', array([ 0.06499979]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.0649998
Name: sequence_1l_438_9_fv1c, dtype: object)
('the mean', array([ 0.13854688]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.138547
Name: sequence_1k_298_9_fv1c, dtype: object)
('the mean', array([ 0.0

Name: sequence_1k_349_10_fv1c, dtype: object)
('the mean', array([ 0.06239086]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.0623909
Name: sequence_1k_349_10_fv1c, dtype: object)
('the mean', array([ 0.13791269]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.137913
Name: sequence_1c_98_9_fv1c, dtype: object)
('the mean', array([ 0.08824184]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0882418
Name: sequence_1c_98_9_fv1c, dtype: object)
('the mean', array([ 157.5457482]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    157.546
Name: sequence_1c_98_9_fv1c, dtype: object)
('the mean', array([ 0.05018102]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the s

("the master df's entry:", mae_avg    0.137838
Name: sequence_1d_128_9_fv1c, dtype: object)
('the mean', array([ 0.08672909]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mse_avg    0.0867291
Name: sequence_1d_128_9_fv1c, dtype: object)
('the mean', array([ 160.91891198]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mape_avg    160.919
Name: sequence_1d_128_9_fv1c, dtype: object)
('the mean', array([ 0.05177003]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", msle_avg    0.05177
Name: sequence_1d_128_9_fv1c, dtype: object)
('the mean', array([ 0.2470255]))
('the type of mean', <class 'pandas.core.series.Series'>)
('the shape of mean', (1,))
("the master df's entry:", mae_avg    0.247025
Name: sequence_2c_548_9_fv1c, dtype: object)
('the mean', array([ 0.21687922]))
('the type of mean'

In [6]:
#create a new df: loadseq vs. score
print(master_per_loadseq_df.index)
print(master_per_loadseq_df.columns)
print(master_per_loadseq_df.head(3))

Index([u'sequence_2c_539_10_fv1c', u'sequence_2c_538_9_fv1c',
       u'sequence_2b_469_10_fv1c', u'sequence_2b_449_10_fv1c',
       u'sequence_1b_19_10_fv1c', u'sequence_2c_518_9_fv1c',
       u'sequence_1c_78_9_fv1c', u'sequence_1b_18_9_fv1c',
       u'sequence_2c_519_10_fv1c', u'sequence_1c_69_10_fv1c',
       ...
       u'sequence_3a_648_10_fv1c', u'sequence_1c_119_10_fv1c',
       u'sequence_1c_118_9_fv1c', u'sequence_1b_58_9_fv1c',
       u'sequence_1b_59_10_fv1c', u'sequence_2c_558_9_fv1c',
       u'sequence_2c_559_10_fv1c', u'sequence_2c_528_9_fv1c',
       u'sequence_2c_529_10_fv1c', u'sequence_3a_569_10_fv1c'],
      dtype='object', name=u'seq_name', length=108)
Index([u'mse_avg', u'mse_med', u'mse_stdev', u'mae_avg', u'mae_med',
       u'mae_stdev', u'mae_normd_dev', u'mape_avg', u'mape_med', u'mape_stdev',
       u'msle_avg', u'msle_med', u'msle_stdev'],
      dtype='object')
                           mse_avg    mse_med mse_stdev   mae_avg    mae_med  \
seq_name            

In [7]:
#load the seqname_loadseqlist csv
#print(master)
complete_sn_lsl_df = pd.read_csv(analysis_path + 'complete_seqname_loadseqlist.csv',index_col=0)
complete_sn_lsl_df.index.rename(name='seq_name',inplace=True)
print(complete_sn_lsl_df.index)
print(complete_sn_lsl_df.columns)

Index([u'sequence_1d_168_9_fv1c', u'sequence_1b_29_10_fv1c',
       u'sequence_1c_119_10_fv1c', u'sequence_1b_8_9_fv1c',
       u'sequence_2c_538_9_fv1c', u'sequence_3a_628_10_fv1c',
       u'sequence_2b_469_10_fv1c', u'sequence_1l_398_9_fv1c',
       u'sequence_1b_49_10_fv1c', u'sequence_3a_687_9_fv1c',
       ...
       u'sequence_1b_38_9_fv1c', u'sequence_1c_68_9_fv1c',
       u'sequence_1d_198_9_fv1c', u'sequence_1k_359_10_fv1c',
       u'sequence_1k_319_10_fv1c', u'sequence_2b_448_9_fv1c',
       u'sequence_1d_229_10_fv1c', u'sequence_1l_428_9_fv1c',
       u'sequence_1d_259_10_fv1c', u'sequence_1l_388_9_fv1c'],
      dtype='object', name=u'seq_name', length=108)
Index([u'loadseq_id', u'loadseq'], dtype='object')


In [8]:
#right df is the one described in the cell above. 
seqname_results_loadseq_df = pd.merge(master_per_loadseq_df,complete_sn_lsl_df,how='inner',left_index=True,right_index=True)
print(seqname_results_loadseq_df.head(2))
print(seqname_results_loadseq_df.columns)
print(seqname_results_loadseq_df.index)

                           mse_avg    mse_med mse_stdev   mae_avg    mae_med  \
seq_name                                                                       
sequence_2c_539_10_fv1c  0.0853845  0.0110813  0.119663   0.13615  0.0115718   
sequence_2c_538_9_fv1c   0.0856342  0.0104185  0.119506  0.136272  0.0117056   

                        mae_stdev mae_normd_dev mape_avg mape_med mape_stdev  \
seq_name                                                                       
sequence_2c_539_10_fv1c  0.180962           NaN  125.079  133.284    42.5061   
sequence_2c_538_9_fv1c   0.180844           NaN  141.869  151.855    43.5794   

                          msle_avg   msle_med msle_stdev  loadseq_id  \
seq_name                                                               
sequence_2c_539_10_fv1c  0.0488543  0.0413555  0.0162035           7   
sequence_2c_538_9_fv1c   0.0488358  0.0413871  0.0161879           7   

                                                                   lo

In [9]:
seqname_results_loadseq_df.to_csv(analysis_path+"seqname_score_loadseq_TEST.csv")

In [10]:
columns_to_cast = list(seqname_results_loadseq_df.columns)
columns_to_cast.remove('loadseq')
print(columns_to_cast)
#seqname_results_loadseq_df['mae_avg'].astype(dtype='float64',copy=False)
#pd.to_numeric([columns_to_cast])
for colname in columns_to_cast: #for some reason it's all objects...gotta cast to numeric. 
    seqname_results_loadseq_df[str(colname)] = pd.to_numeric(seqname_results_loadseq_df[str(colname)])
for colname in columns_to_cast:
    print("new", colname, seqname_results_loadseq_df[colname].dtype)
grouped_srls_df = seqname_results_loadseq_df.groupby(by=['loadseq']).mean()
#print(grouped_srls_df['mae_avg'].mean())
#print(grouped_srls_df.median())
#grouped.srls.df.reset_index()
print(grouped_srls_df.head(29))
print(type(grouped_srls_df))
#print(type(grouped_srls_df.groups))
#print(grouped_srls_df.head(1))#print(grouped_srls_df.mae_avg)
#print(grouped_srls_df.agg(np.mean))
#for key,value in grouped_srls_df.groups.iteritems():
    #print(value)
#print(grouped_srls_df.head(2))
pd.DataFrame(grouped_srls_df).to_csv(analysis_path+'srls_grouped_by_ls.csv')

['mse_avg', 'mse_med', 'mse_stdev', 'mae_avg', 'mae_med', 'mae_stdev', 'mae_normd_dev', 'mape_avg', 'mape_med', 'mape_stdev', 'msle_avg', 'msle_med', 'msle_stdev', 'loadseq_id']
('new', 'mse_avg', dtype('float64'))
('new', 'mse_med', dtype('float64'))
('new', 'mse_stdev', dtype('float64'))
('new', 'mae_avg', dtype('float64'))
('new', 'mae_med', dtype('float64'))
('new', 'mae_stdev', dtype('float64'))
('new', 'mae_normd_dev', dtype('float64'))
('new', 'mape_avg', dtype('float64'))
('new', 'mape_med', dtype('float64'))
('new', 'mape_stdev', dtype('float64'))
('new', 'msle_avg', dtype('float64'))
('new', 'msle_med', dtype('float64'))
('new', 'msle_stdev', dtype('float64'))
('new', 'loadseq_id', dtype('int64'))
                                                     mse_avg   mse_med  \
loadseq                                                                  
[  1.028  10.28   10.28    0.75    0.75   10.28...  0.085509  0.010750   
[  1.028  11.28   11.28    1.      1.     10.28...  0.163782 

In [11]:
grouped_srls_count_df = seqname_results_loadseq_df.groupby(by=['loadseq']).count()
pd.DataFrame(grouped_srls_count_df).to_csv(analysis_path+'srls_grouped_by_ls_count.csv')

In [12]:
#group by initial conditions. 
